In [2]:
from geopy.geocoders import Nominatim

# Nominatim = service gratuit basé sur OpenStreetMap
geolocator = Nominatim(user_agent="geoapi")

In [16]:
import ndjson
import geopandas as gpd
from shapely.geometry import Point


# Chemin vers ton fichier
fichier_ndjson = "../data/adresses/adresses-addok-27.ndjson"

# Lire le NDJSON compressé
with open(fichier_ndjson, 'rt', encoding='utf-8') as f:
    data = ndjson.load(f)
records = []


for item in data:
    if "geometry" in item and item["geometry"] is not None:
        coords = item["geometry"]["coordinates"]  # [lon, lat]
        properties = item.get("properties", {})
        properties["geometry"] = Point(coords)  # créer géométrie explicitement
        records.append(properties)

if not records:
    raise ValueError("Aucune donnée valide trouvée dans le fichier NDJSON.")

# Créer GeoDataFrame
gdf = gpd.GeoDataFrame(records, geometry="geometry", crs="EPSG:4326")

# Reprojection vers Lambert-93
gdf = gdf.to_crs(epsg=2154)


ValueError: Aucune donnée valide trouvée dans le fichier NDJSON.

In [ ]:
from functools import partial
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="sirene")


In [5]:
geocode = partial(geolocator.geocode, language="fr")

In [8]:
location = geolocator.geocode("48 RUE DES COTEAUX DE PACY, PACY SUR EURE")

In [9]:
location

Location(48, Rue des Coteaux de Pacy, Pacy-sur-Eure, Les Andelys, Eure, Normandie, France métropolitaine, 27120, France, (49.0167283, 1.390151, 0.0))

In [12]:
location.address

'48, Rue des Coteaux de Pacy, Pacy-sur-Eure, Les Andelys, Eure, Normandie, France métropolitaine, 27120, France'

In [10]:
location.latitude

49.0167283

In [11]:
location.longitude

1.390151

In [13]:
!pip install requests

In [18]:
import requests
ADDOK_URL = 'http://api-adresse.data.gouv.fr/search/'
params = {
    'q': location.address,
    'limit': 5
}
response = requests.get(ADDOK_URL, params=params)
j = response.json()
if len(j.get('features')) > 0:
    first_result = j.get('features')[0]
    lon, lat = first_result.get('geometry').get('coordinates')
    first_result_all_infos = { **first_result.get('properties'), **{"lon": lon, "lat": lat}}
    print(first_result_all_infos)
else:
    print('No result')

{'label': "31 Rue d'Alembert 02100 Saint-Quentin", 'score': 0.34616827890556046, 'housenumber': '31', 'id': '02691_0115_00031', 'banId': 'aea56212-ec7e-4c7b-abf2-e007220b5d28', 'name': "31 Rue d'Alembert", 'postcode': '02100', 'citycode': '02691', 'x': 721307.38, 'y': 6973973.35, 'city': 'Saint-Quentin', 'context': '02, Aisne, Hauts-de-France', 'type': 'housenumber', 'importance': 0.66222, 'street': "Rue d'Alembert", 'lon': 3.296125, 'lat': 49.864165}
